In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import pandas as pd, time
from bs4 import BeautifulSoup

<h1>Review Scraper</h1>
<br>
<div>Running this notebook will scrape all reviews in the google play store for the app specified in the variable app_id and dump the reviews, including reviewer name, rating, date, upvotes and text in a csv. We ran the scraper for popular European public transportation apps in order to elicit requirements for our own app.</div>

In [2]:
#specify name of app here
app_name="amsterdamGVB"
#specify app id in google play store here
app_id="nl.moopmobility.gvb"

In [3]:
url="https://play.google.com/store/apps/details?id="+app_id+"&hl=en&showAllReviews=true"

In [4]:
option=webdriver.ChromeOptions()
option.add_argument(" - incognito")
browser=webdriver.Chrome(executable_path='chromedriver',chrome_options=option)

<ipython-input-4-c8aa7650c419>:3: DeprecationWarning: use options instead of chrome_options
  browser=webdriver.Chrome(executable_path='chromedriver',chrome_options=option)


In [5]:
browser.get(url)

In [6]:
def scroll(driver, timeout):

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(timeout)
                    
        #check for show more
        try:
            button_path=driver.find_element_by_class_name("PFAhAf").find_element_by_tag_name("div")
            ActionChains(driver).click(button_path).perform()
            print("Button to 'show more' clicked. Loading more reviews...")
            time.sleep(timeout)
        except:
            print("Scrolled to bottom. Loading more reviews...")
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            print("All reviews loaded!")
            break
        last_height = new_height

In [7]:
scroll(browser,3)

Scrolled to bottom. Loading more reviews...
Scrolled to bottom. Loading more reviews...
All reviews loaded!


In [8]:
review_path=browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div"}
  (Session info: chrome=84.0.4147.125)


In [9]:
reviews=review_path.find_elements_by_class_name("zc7KVe")

In [10]:
print("Number of reviews: "+str(len(reviews)//2))

Number of reviews: 69


In [11]:
review_array=[]

In [12]:
for review in reviews[::2]:
    name=review.find_element_by_class_name("X43Kjb").get_attribute('innerHTML')
    print(name)
    star_path=review.find_element_by_class_name("pf5lIe").find_element_by_tag_name('div')
    star=len(star_path.find_elements_by_class_name("vQHuPe"))
    print(star)
    date=review.find_element_by_class_name("p2TkOb").get_attribute('innerHTML')
    print(date)
    helpful_path=review.find_element_by_class_name("XlMhZe")
    helpful=helpful_path.find_elements_by_tag_name("div")[-1].get_attribute('innerHTML')
    print(helpful)
    review_elements=review.find_element_by_class_name("UD7Dzf").find_elements_by_tag_name("span")
    review_text=review_elements[-1].get_attribute('innerHTML')
    if(len(review_text)==0):
        review_text=review_elements[0].get_attribute('innerHTML')
    print(review_text)
    review_array.append([name,star,date,helpful,review_text])

A Google user
3
September 30, 2018
20
Confusing for tourists with tourist tickets that are restricted to lines operated by certain companies. The app doesn't give information about the operators. I ended up with a multi-stop connection where the bus driver refused my ticket and I had to travel back to take a different connection.
Ozgur Kalkan
3
November 19, 2018
4
It's a good app, more like an essential one, but the trams and trains always change routes and there is always something wrong resulting in me either getting late or not getting there at all.
gabs igab
5
November 7, 2019
2
Excellent app I have used it on our visit to Amsterdam and was excellent. Great help to have with you. Good info, accurate, reliable. Good planner also.
Damian
5
March 1, 2020
1
It works very well for traveling. I use it everyday.
Pel Pan
1
March 28, 2019
1
very bad accuracy and often not the shortest way
Dinesh VG
1
October 3, 2018
2
Super confusing app!
Paulo Waisberg
5
July 24, 2019
1
We will see so far 

In [13]:
df=pd.DataFrame(review_array,columns=["name","rating","date","upvotes","review"])

In [14]:
df.to_csv(app_name+'.csv')